In [161]:
import luigi
import pandas as pd
import os


## Création du dossier pour stocker les données nettoyées

In [162]:
# Vérifiez si le dossier n'existe pas déjà

dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    print("Dossier créé avec succès.")


In [163]:
class ReadDataCanton(luigi.Task):
    file_path = luigi.Parameter()


    def output(self):
        return luigi.LocalTarget("./CleanData/canton.csv")

    def run(self):
        
        df = pd.read_csv(self.file_path, sep=";")
        cantons = pd.DataFrame(columns=df.columns[0:25])
        df_partis = pd.read_csv('./RawData/Partis.csv', sep=",")

        for index, row in df.iterrows():
            #print(f"index : {index} \n row : {row[0:18]} \n\n")
            row1 = pd.Series(row[0:18], index=df.columns[0:18])

            for i in range(0, 10):
                # Ligne 2
                row2 = pd.Series(row.iloc[18 + i * 7:18 + ((i + 1) * 7)])

                df_row2 = pd.DataFrame([row2])
                
                df_row2_renamed = pd.DataFrame(df_row2.values, columns=['N°Panneau','Sexe','Nom','Prenom','Voix','% Voix/Ins','% Voix/Exp'])

                # Fusionner les deux lignes
                merged_row = pd.concat([row1, df_row2_renamed.squeeze()], axis=0)

                # Convertir la ligne fusionnée en DataFrame avec une seule ligne
                merged_row_df = pd.DataFrame([merged_row])

                # Ajouter la ligne fusionnée au DataFrame
                cantons = pd.concat([cantons, merged_row_df], ignore_index=True)

                #print(len(cantons))

        cantons = pd.merge(cantons, df_partis, on=['Nom', 'Prenom'], how='left')

        print(cantons.duplicated(['Nom', 'Prenom']).sum())
        print(df_partis.duplicated(['Nom', 'Prenom']).sum())

        #display(cantons)
        cantons.to_csv(self.output().path, index=False)




## Extraire les candidats

In [164]:
class ExtractCandidats(luigi.Task):
    file_path = luigi.Parameter()

    def requires(self):
        return ReadDataCanton(file_path="./RawData/canton_t1_2017.csv")
    
    def output(self):
        return luigi.LocalTarget("./CleanData/candidats.csv")

    def run(self):
                
        df = pd.read_csv(self.file_path, sep=",")
       
        display(df)

        df = df.loc[:, ["Prenom", "Nom"]]
        df = df.drop_duplicates()
        
        df.to_csv(self.output().path, index=False)

## Extraire les données sur la délinquance

In [165]:
class ExtractDelinquance(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/delinquance.csv")

    def run(self):
                        
        df = pd.read_csv(self.file_path, sep=";")
        

        df = df.query('annee == 17')

        df = df.rename(columns={"classe" : "Type", "Code.département" : "Code du departement", "Code.région" : "Region"})

        grouped_df = df.groupby('Code du departement')['faits'].sum()

        grouped_df = grouped_df.reset_index()

        grouped_df.to_csv(self.output().path, index=False)

## Calculer le taux de chômage par département

In [166]:
class ExtractNombreChomeurs(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/chomeurs.csv")

    def run(self):      
        df = pd.read_csv(self.file_path, sep=";")
        df = df.rename(columns={"Code Officiel Département" : "Code du departement"})

        df = df[df['Année'] == 2017]

        df_chomeurs = df[df['Nom de la variable'].str.contains('Chômeurs')]
        df_chomeurs = df_chomeurs.groupby('Code du departement')['Valeur'].sum()
        df_chomeurs = df_chomeurs.reset_index()


        df_actifs = df[df['Nom de la variable'].str.contains('Actifs')]
        df_actifs = df_actifs.groupby('Code du departement')['Valeur'].sum()
        df_actifs = df_actifs.reset_index()

        df_merge = pd.merge(df_chomeurs, df_actifs, on='Code du departement')
        df_merge["Chomage"] = df_merge["Valeur_x"] / df_merge["Valeur_y"]
        

        df_merge.to_csv(self.output().path, index=False)


## Exécuter toutes les tâches

In [169]:
class ReadAllData(luigi.Task):
    def requires(self):
        return [ExtractCandidats(file_path="./CleanData/canton.csv"), 
                ExtractDelinquance(file_path="./RawData/Donnees_delinquance.csv"),
                ExtractNombreChomeurs(file_path="./RawData/Population active par département.csv")]
    
    def run(self):
        print("lancement")

    def output(self):
        return luigi.LocalTarget('result.txt')

In [170]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')

dossier = "./CleanData/"
Restart = False

if Restart is True:
    # Parcourir tous les fichiers du dossier
    for fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, fichier)
        # Supprimer le fichier
        os.remove(chemin_fichier)

luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)

DEBUG: Checking if ReadAllData() is complete
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\luigi\worker.py", line 877, in _add
    deps = task.deps()
           ^^^^^^^^^^^
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\luigi\task.py", line 676, in deps
    return flatten(self._requires())
                   ^^^^^^^^^^^^^^^^
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\luigi\task.py", line 648, in _requires
    return flatten(self.requires())  # base impl
                   ^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_17904\3156948128.py", line 5, in requires
    CalculerTauxChomage(file_path="./RawData/Population active par département.csv")]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\luigi\task_register.py", line 87, 

False